In [ ]:
# ================================================================
# STEP 1–4: Omics Integration (CCLE + GDSC + Mutations → Reference Drug)
# ================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns
import os

# ===========================
# Load CCLE expression
# ===========================
expr_file = "CCLE_expression.csv"
info_file = "sample_info.csv"
if not os.path.exists(expr_file) or not os.path.exists(info_file):
    raise FileNotFoundError("Download CCLE_expression.csv and sample_info.csv from DepMap")

df_expr = pd.read_csv(expr_file)
KRAS_cols = [c for c in df_expr.columns if "KRAS" in c.upper()]
if not KRAS_cols:
    raise ValueError("KRAS column not found in expression file")
KRAS_col = KRAS_cols[0]
id_col = "Unnamed: 0"
expr = df_expr[[id_col, KRAS_col]].copy()
expr.columns = ["cell_line_id", "KRAS_expr"]

# Merge expression with metadata
info = pd.read_csv(info_file)
merged_expr = expr.merge(info, left_on="cell_line_id", right_on="ModelID", how="left")

# Filter only colon/colorectal lines
crc_diseases = ["Colon", "Colorectal"]
crc_expr = merged_expr[
    merged_expr["OncotreePrimaryDisease"].str.contains('|'.join(crc_diseases), case=False, na=False)
].copy()
crc_expr_sorted = crc_expr.sort_values("KRAS_expr", ascending=False)
crc_expr_sorted.to_csv("KRAS_high_crc.csv", index=False)

# ===========================
# Harmonize GDSC data
# ===========================
ic50_file = "GDSC2_fitted_dose_response.csv"
cell_info_file = "Cell_Lines_Details.csv"
if not os.path.exists(ic50_file) or not os.path.exists(cell_info_file):
    raise FileNotFoundError("Download GDSC2_fitted_dose_response.csv and Cell_Lines_Details.csv")

ic50 = pd.read_csv(ic50_file)
cell_info = pd.read_csv(cell_info_file)
cell_info.rename(columns={'COSMIC identifier': 'COSMIC_ID',
                          'Sample Name': 'CellLineName'}, inplace=True)

# Clean and align IDs
cell_info = cell_info.dropna(subset=['COSMIC_ID']).copy()
cell_info['COSMIC_ID'] = cell_info['COSMIC_ID'].astype(float).astype(int)
crc_expr_sorted = crc_expr_sorted.dropna(subset=['COSMICID']).copy()
crc_expr_sorted['COSMICID'] = crc_expr_sorted['COSMICID'].astype(float).astype(int)
ic50 = ic50.dropna(subset=['COSMIC_ID']).copy()
ic50['COSMIC_ID'] = ic50['COSMIC_ID'].astype(int)

# Merge CRC expression → GDSC info
merged_crc = crc_expr_sorted.merge(
    cell_info[['COSMIC_ID', 'CellLineName']],
    left_on='COSMICID', right_on='COSMIC_ID', how='inner'
)
print("CRC lines mapped:", merged_crc['cell_line_id'].nunique())

# Subset IC50 for mapped CRC lines
ic50_crc = ic50[ic50['COSMIC_ID'].isin(merged_crc['COSMIC_ID'])].copy()
ic50_crc.to_csv("CRC_ic50_intersection.csv", index=False)

# ===========================
# Merge expression + IC50 for selected drugs
# ===========================
expr_small = merged_crc[['cell_line_id', 'KRAS_expr', 'COSMICID']].copy()
selected_drugs = ["Afatinib", "Gefitinib", "Refametinib", "Navitoclax", "PLX-4720",
                  "Trametinib", "Selumetinib", "Sotorasib", "Adagrasib"]

ic50_selected = ic50_crc[ic50_crc['DRUG_NAME'].isin(selected_drugs)].copy()
merged_multi = pd.merge(
    expr_small,
    ic50_selected[['COSMIC_ID', 'DRUG_NAME', 'LN_IC50']],
    left_on='COSMICID', right_on='COSMIC_ID', how='inner'
)
merged_multi.to_csv("KRAS_expr_multi_drugs.csv", index=False)

# ===========================
# Add KRAS mutation data
# ===========================
mut_file = "CCLE_mutations.csv"
if not os.path.exists(mut_file):
    raise FileNotFoundError("Download CCLE_mutations.csv from DepMap")

df_mut = pd.read_csv(mut_file)
kras_mut = df_mut[df_mut['Hugo_Symbol'].str.upper() == "KRAS"]
cols = ['DepMap_ID', 'Hugo_Symbol', 'Protein_Change', 'Variant_Classification',
        'Chromosome', 'Start_position', 'End_position', 'Variant_Type']
kras_mut = kras_mut[cols]

merged_with_mut = merged_multi.merge(kras_mut, left_on="cell_line_id", right_on="DepMap_ID", how="left")
merged_with_mut.to_csv("KRAS_expr_ic50_mut.csv", index=False)

# ===========================
# Correlation analysis
# ===========================
df_corr = merged_with_mut.dropna(subset=["KRAS_expr", "LN_IC50"])
results = []
for drug in df_corr['DRUG_NAME'].unique():
    sub = df_corr[df_corr['DRUG_NAME'] == drug]
    if len(sub) > 5:
        r, p = pearsonr(sub['KRAS_expr'].astype(float), sub['LN_IC50'].astype(float))
        results.append({"drug": drug, "pearson_r": r, "p_value": p, "n": len(sub)})

results_df = pd.DataFrame(results).sort_values("pearson_r")
results_df.to_csv("drug_correlation_results.csv", index=False)
print("Correlation results saved: drug_correlation_results.csv")
print(results_df.head())

# ===========================
# Reference drug selection
# ===========================
reference_drug = results_df.iloc[0]['drug']
kras_pathway_drugs = ["Sotorasib", "Adagrasib", "Trametinib", "Selumetinib"]
if reference_drug not in kras_pathway_drugs:
    available = [d for d in kras_pathway_drugs if d in df_corr['DRUG_NAME'].unique()]
    if available:
        print(f"Overriding to biologically relevant drug: {available[0]}")
        reference_drug = available[0]
print("Final chosen reference drug:", reference_drug)

# ===========================
# Visualization
# ===========================
for drug in df_corr['DRUG_NAME'].unique():
    sub = df_corr[df_corr['DRUG_NAME'] == drug]
    if len(sub) > 5:
        x = sub['KRAS_expr'].astype(float)
        y = sub['LN_IC50'].astype(float)
        r, p = pearsonr(x, y)
        plt.figure(figsize=(5,4))
        plt.scatter(x, y, alpha=0.7)
        plt.xlabel("KRAS expression (log2 TPM+1)")
        plt.ylabel("LN_IC50")
        plt.title(f"{drug}: r={r:.2f}, p={p:.2e}")
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.tight_layout()
        plt.savefig(f"{drug}_expr_vs_ic50.png", dpi=200)
        plt.close()


In [ ]:
#step 5 the scrna wala step
# =========================
# Install dependencies (only first time)
# =========================
# !pip install scanpy==1.9.3 anndata==0.8.0 matplotlib seaborn pandas scipy

# =========================
# Imports
# =========================
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
import os
from scipy.io import mmread

# =========================
# File paths (adjust if needed)
# =========================
matrix_file = "matrix.mtx"
genes_file = "genes.csv"      # or features.tsv
barcodes_file = "barcodes.csv"
clinical_file = "table.csv"   # <-- using CSV now

# =========================
# Load metadata
# =========================
genes = pd.read_csv(genes_file, header=None, sep=None, engine="python")
genes.columns = ["gene_id", "gene_symbol"]

barcodes = pd.read_csv(barcodes_file, header=None)
barcodes.columns = ["cell_id"]

# =========================
# Load matrix (MTX)
# =========================
X = mmread(matrix_file).tocsr()
print("Matrix shape (cells x genes):", X.shape)

# =========================
# Auto-fix mismatches
# =========================
# Genes
if X.shape[1] != genes.shape[0]:
    print(f"⚠️ Gene mismatch: matrix has {X.shape[1]} genes, file has {genes.shape[0]}")
    min_len = min(X.shape[1], genes.shape[0])
    X = X[:, :min_len]
    genes = genes.iloc[:min_len, :]
    print("✅ Fixed genes ->", min_len)

# Barcodes
if X.shape[0] != barcodes.shape[0]:
    print(f"⚠️ Barcode mismatch: matrix has {X.shape[0]} cells, file has {barcodes.shape[0]}")
    min_len = min(X.shape[0], barcodes.shape[0])
    X = X[:min_len, :]
    barcodes = barcodes.iloc[:min_len, :]
    print("✅ Fixed barcodes ->", min_len)

# =========================
# Create AnnData
# =========================
adata = sc.AnnData(X)
adata.var['gene_symbol'] = genes['gene_symbol'].values
adata.obs['cell_id'] = barcodes['cell_id'].values

adata.obs.set_index('cell_id', inplace=True)
adata.var.set_index('gene_symbol', inplace=True)

# =========================
# Merge clinical metadata (optional)
# =========================
if os.path.exists(clinical_file):
    clinical = pd.read_csv(clinical_file)
    adata.obs['Patient_ID'] = [c.split("_")[1] for c in adata.obs_names]
    adata.obs = adata.obs.merge(clinical, how='left',
                                left_on='Patient_ID', right_on='Patient ID')
    print("✅ Clinical metadata merged.")
else:
    print("⚠️ Clinical file not found, skipping merge.")

# =========================
# Basic QC
# =========================
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)

# =========================
# Normalization & HVG
# =========================
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]

# =========================
# PCA, neighbors, UMAP, clustering
# =========================
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata)

# =========================
# Save plots (optimized for speed)
# =========================
sc.pl.umap(adata, color=['leiden'], size=5, dpi=80, show=False, save='_clusters.png')

if 'KRAS' in adata.var_names:
    sc.pl.umap(adata, color=['KRAS'], size=5, dpi=80, show=False, save='_kras.png')
else:
    print("⚠️ KRAS gene not found. Check var_names:", adata.var_names[:10])


# =========================
# Save processed AnnData
# =========================
adata.write("GSE178318_processed.h5ad")
print("✅ Saved processed AnnData: GSE178318_processed.h5ad")


In [ ]:
# ================================================
# 📘 Step 6: ChEMBL Data Processing Pipeline
# From raw target CSVs → cleaned, standardized dataset
# Author: [Your Name]
# ================================================

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys

# ================================================
# Step 1: Define Input Files (ChEMBL Downloads)
# ================================================
files = {
    "SOS1-KRAS": r"C:\Users\HomePC\Desktop\lab\MTDLs\asinex\chem\CHEMBL5465393 (SOS1-KRAS).csv",
    "KRAS": r"C:\Users\HomePC\Desktop\lab\MTDLs\asinex\chem\CHEMBL2189121 (GTPase KRas).csv",
    "MEK": r"C:\Users\HomePC\Desktop\lab\MTDLs\asinex\chem\CHEMBL2111351.csv"
}

all_dfs = []

# ================================================
# Step 2: Clean and Standardize Each File
# ================================================
for target, file in files.items():
    df = pd.read_csv(file, sep=";")

    # Keep key columns (adjust if names differ)
    keep = [
        "Molecule ChEMBL ID", "Smiles", "Molecular Weight",
        "AlogP", "#RO5 Violations", "Standard Type",
        "Value", "Standard Units"
    ]
    df = df[[c for c in keep if c in df.columns]].copy()
    df = df[df["Value"].notna()]
    df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
    df = df.dropna(subset=["Value"])

    # Convert all IC50/Activity values to nM
    def to_nM(row):
        unit = str(row["Standard Units"]).lower()
        val = row["Value"]
        if unit == "nm":
            return val
        elif unit == "µm" or unit == "um":
            return val * 1_000
        elif unit == "mm":
            return val * 1_000_000
        else:
            return np.nan

    df["IC50_nM"] = df.apply(to_nM, axis=1)
    df = df.dropna(subset=["IC50_nM"])

    # Label by activity thresholds
    df["Activity_Class"] = "Intermediate"
    df.loc[df["IC50_nM"] <= 1000, "Activity_Class"] = "Active"
    df.loc[df["IC50_nM"] >= 10000, "Activity_Class"] = "Inactive"

    # Add Target info
    df["Target"] = target

    all_dfs.append(df)

# ================================================
# Step 3: Merge All Targets
# ================================================
combined = pd.concat(all_dfs, ignore_index=True)
combined.to_csv("chembl_combined_clean.csv", index=False)

print("✅ Saved → chembl_combined_clean.csv")
print("Shape:", combined.shape)
print("Counts per target:\n", combined["Target"].value_counts())
print("Activity class distribution:\n", combined["Activity_Class"].value_counts())

# ================================================
# Step 4: Convert IC50 → pIC50 and Apply Lipinski Rules
# ================================================
df = combined.copy()
df["pIC50"] = 9 - np.log10(df["IC50_nM"])
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["pIC50"])

# Compute Lipinski descriptors
def lipinski(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return pd.Series([np.nan] * 5)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    ro5 = sum([mw > 500, logp > 5, hbd > 5, hba > 10])
    return pd.Series([mw, logp, hbd, hba, ro5])

df[["MW", "LogP", "HBD", "HBA", "RO5"]] = df["Smiles"].apply(lipinski)

# ================================================
# Step 5: Reassign Activity Classes by pIC50
# ================================================
def label_activity(pIC50):
    if pIC50 >= 6:
        return "Active"
    elif pIC50 <= 5:
        return "Inactive"
    else:
        return "Intermediate"

df["Activity_Class"] = df["pIC50"].apply(label_activity)

# ================================================
# Step 6: Generate Molecular Fingerprints
# ================================================
def morgan_fp(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) if mol else None

def maccs_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return MACCSkeys.GenMACCSKeys(mol) if mol else None

df["MorganFP"] = df["Smiles"].apply(morgan_fp)
df["MACCSFP"] = df["Smiles"].apply(maccs_fp)

# ================================================
# Step 7: Save Final Processed Dataset
# ================================================
df.to_csv("chembl_processed.csv", index=False)

print("\n✅ Final dataset saved → chembl_processed.csv")
print("Final shape:", df.shape)
print("Activity class balance:\n", df["Activity_Class"].value_counts())


In [ ]:
# ================================
# ML + Selumetinib similarity integration
# ================================

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# ================================
# Step 1: Load datasets
# ================================
df = pd.read_csv("chembl_processed.csv")
similarity = pd.read_csv("selumetinib_similarity_filtered.csv")

# Keep only Active vs Inactive (drop intermediates)
df = df[df["Activity_Class"].isin(["Active", "Inactive"])].copy()
df["y"] = df["Activity_Class"].map({"Active": 1, "Inactive": 0})

print("Dataset shape:", df.shape)

# ================================
# Step 2: Fingerprint generation
# ================================
def mol_to_fps(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None
    morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    maccs = MACCSkeys.GenMACCSKeys(mol)
    return np.array(morgan), np.array(maccs)

fps_morgan, fps_maccs = [], []
for smi in df["Smiles"]:
    m, a = mol_to_fps(smi)
    if m is None:
        fps_morgan.append([0]*2048)
        fps_maccs.append([0]*167)
    else:
        fps_morgan.append(m)
        fps_maccs.append(a)

X_morgan = np.array(fps_morgan)
X_maccs = np.array(fps_maccs)
y = df["y"].values

# ================================
# Step 3: Define models
# ================================
models = {
    "RandomForest": RandomForestClassifier(n_estimators=200, random_state=42),
    "SVM": SVC(kernel="rbf", C=1, gamma=0.1, probability=True, random_state=42),
    "ANN": MLPClassifier(hidden_layer_sizes=(128, 64), activation="relu",
                         solver="adam", max_iter=200, random_state=42)
}

# ================================
# Step 4: Cross-validation & evaluation
# ================================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
performance = []

preds_all = pd.DataFrame({"Molecule ChEMBL ID": df["Molecule ChEMBL ID"], "True_Label": y})

for name, model in models.items():
    preds = cross_val_predict(model, X_morgan, y, cv=skf, method="predict_proba")[:, 1]
    y_pred = (preds >= 0.5).astype(int)

    auc = roc_auc_score(y, preds)
    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred)
    rec = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)

    performance.append([name, auc, acc, prec, rec, f1])

    preds_all[f"{name}_prob"] = preds

perf_df = pd.DataFrame(performance, columns=["Model", "AUC", "Accuracy", "Precision", "Recall", "F1"])
perf_df.to_csv("ml_performance.csv", index=False)
preds_all.to_csv("ml_predictions.csv", index=False)

print("\nML Performance:")
print(perf_df)

# ================================
# Step 5: Merge with Selumetinib similarity
# ================================
merged = preds_all.merge(similarity, on="Molecule ChEMBL ID", how="inner")

# Use RF as main model for ranking
merged["Rank_Score"] = merged["RandomForest_prob"] * (
    merged["Morgan_Similarity"] + merged["MACCS_Similarity"]
) / 2

# Sort best to worst
top_hits = merged.sort_values("Rank_Score", ascending=False)
top_hits.to_csv("top_hits_pre_docking.csv", index=False)

print("\n✅ Final shortlist saved → top_hits_pre_docking.csv")
print(top_hits.head(10))


In [ ]:
# =========================================
# Step 1: Load top hits to filter by criteria
# =========================================
import pandas as pd

hits = pd.read_csv("top_hits_pre_docking.csv")

# =========================================
# Step 2: Apply strict consensus filters
# =========================================
filtered = hits[
    (hits["Rank_Score"] >= 0.35) &
    (hits["RandomForest_prob"] >= 0.95) &
    (hits["SVM_prob"] >= 0.85) &
    (hits["ANN_prob"] >= 0.99) &
    (hits["Activity_Class"] == "Active")   # keep only experimental actives
].copy()

print("Filtered consensus hits:", filtered.shape[0])

# =========================================
# Step 3: Save for docking
# =========================================
filtered.to_csv("consensus_hits_for_docking.csv", index=False)

print("\n✅ Saved → consensus_hits_for_docking.csv")
print(filtered.head(10))
